# Inputs

In [ ]:
from PIL import Image

input_img = Image.open("images/Image_3.jpg")

# Implementation

In [ ]:
from transformers import pipeline
import torch

objectDetectionModel = pipeline(task = "object-detection", model = "facebook/detr-resnet-50")

objectsDetected = objectDetectionModel(input_img)

# feature extraction from object detection output.

label_counts = {}
for item in objectsDetected:
    label = item['label']
    label_counts[label] = label_counts.get(label, 0) + 1

labels = (label_counts.keys())

textModelInput = "There are "

for item in labels:
    count_label = label_counts[item]
    textModelInput += str(count_label)
    textModelInput += " "
    textModelInput += item + ","

textModelInput += " in this image."

# text summarization from feature extraction output.

# Specify the inference task
generator = pipeline(
    task="text-generation",
    model="Qwen/Qwen3-0.6B",
    torch_dtype=torch.bfloat16,
)

response = generator(textModelInput, max_length=200)

inputforAudioModel = ""
for value in response:
    allKeys = value.keys()
    for key in allKeys:
        inputforAudioModel = value[key]

# passing text generator output to audio model.

narrator = pipeline(
    task="text-to-speech",
    model="suno/bark-small",
)

naratorOutput = narrator(inputforAudioModel);



# Output

In [ ]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(
    naratorOutput["audio"],
    rate=naratorOutput["sampling_rate"]
)